# ML System Design: Case Study on eBay Auction System

## Table of Contents
1. [Case Study Overview](#case-study-overview)  
2. [Functional Requirements](#functional-requirements)  
3. [Non-Functional Requirements](#non-functional-requirements)  
4. [Core Entities in the System](#core-entities-in-the-system)  
5. [API Design](#api-design)  
6. [High-Level Design](#high-level-design)  
7. [Fault Tolerance and Durability](#fault-tolerance-and-durability)  
8. [Concurrency and Consistency](#concurrency-and-consistency)  
9. [Locking Mechanisms](#locking-mechanisms)  
10. [Utilizing Redis for Caching](#utilizing-redis-for-caching)  
11. [Trade-offs of System Design](#trade-offs-of-system-design)  
12. [Schema Design](#schema-design)  
13. [Example SQL Code](#example-sql-code)  
14. [Flowchart Illustration](#flowchart-illustration)  
15. [Full Flowchart of Ebay Auction system](#full-flowchart-of-ebay-auction-system)
16. [Summary](#summary)  

---

## Case Study Overview
This case study investigates the online bidding system akin to eBay, designed to facilitate product auctions. It incorporates two primary users: sellers who list items for auction and buyers who submit bids. The notes explore core requirements, system design, and inherent challenges.

## Functional Requirements
1. **Post an Item for Auction by Seller**
   - Sellers must provide:
     - Starting price (e.g., 12 Rs)
     - End time for the auction
     - Product details (description, images, etc.).

2. **Post a Bid on Auction Item by Buyer**
   - Bids must exceed the current highest bid.
   - Example: If the highest bid is 12 Rs, only bids of 13 Rs or higher will be accepted.

3. **View Highest Bidding Price**
   - Users should dynamically view the real-time highest bid price.

**Out of Scope:**
- Searching the catalog.
- Auction history.
- Payment services integration.

## Non-Functional Requirements
1. **High Consistency:** Ensuring all users see the latest bid data accurately.
2. **Fault Tolerance and Durability:** The system must retain all bids and recover from failures.
3. **Scalability:** Supporting up to 10 million concurrent users.
4. **Low Latency for High Bid Views:** Instant visibility of the current highest bid.

**Out of Scope:**
- Monitoring.
- Observability.
- Security.

---

## Core Entities in the System
### Key Tables
1. **Auction Table**
   - Attributes include:
     - `id`
     - `item_id`
     - `start_date`
     - `end_date`
     - `starting_price`
     - `user_id` (seller's ID)

2. **Bid Table**
   - Attributes include:
     - `id`
     - `auction_id`
     - `user_id` (bidder's ID)
     - `bid_price`
     - `created_at`
     - `status` (accepted or rejected)

3. **Item Table**
   - Attributes include:
     - `id`
     - `name`
     - `description`
     - `image_links`

### Rationale for Separate Item Table
A separate item table allows for easier modifications to item details without impacting ongoing auctions.

---

## API Design
### Key API Endpoints
1. **Post Auction Item**
   - **Request Parameters:**
     - `item_id`
     - `start_date`
     - `end_date`
     - `starting_price`
   - **Response:** Auction object and item object.

2. **Post Bid on Auction Item**
   - **Request Parameters:**
     - `auction_id`
     - `bid_price`
     - `created_at` timestamp
   - **Response:** Status of the bid.

3. **Get View of Bid**
   - **Request:** `auction_id`
   - **Response:** Details of the current highest bid.

---

## High-Level Design
### Functional Flow
1. **Client Interaction:**
   Clients connect through an API Gateway that routes to the respective services (auction or bid).

```
Client --> API Gateway --> Auction Service --> Database
                ---> Bid Service --> Database
```

2. **Database Structure:**
   Tables for Auctions, Items, and Bids facilitate efficient data handling.

### Auction Table Example
| Column Name       | Data Type  | Description                      |
|-------------------|------------|----------------------------------|
| id                | INT        | Primary key                      |
| item_id           | INT        | ID referencing the item          |
| start_date        | DATETIME   | Auction starting date            |
| end_date          | DATETIME   | Auction ending date              |
| starting_price    | DECIMAL    | Initial bid price                |
| user_id           | INT        | Seller's user ID                 |

### Item Table Example
| Column Name       | Data Type  | Description                                  |
|-------------------|------------|----------------------------------------------|
| id                | INT        | Primary key                                  |
| name              | VARCHAR    | Name of the product                          |
| description       | TEXT       | Description of the product                   |
| image_links       | TEXT       | Links to product images                      |

---

## Fault Tolerance and Durability
### Fault Tolerance
**Definition**: The ability of a system to continue functioning correctly in the event of a failure.

- **Key Principles**:
  - **Redundancy**: Backup components to manage system failures.
  - **Graceful Degradation**: System performance should gracefully reduce instead of failing completely.
  - **Failover Mechanisms**: Automatically switch to standby components when a failure occurs.

### Durability
**Definition**: Ensuring data and models are preserved over time, even in failures.

- **Key Principles**:
  - **Persistent Storage**: Utilize reliable storage solutions (e.g., Amazon S3).
  - **Replication**: Maintain multiple data copies across different locations.
  - **Versioning and Backups**: Ensure that various data versions exist for rollback capability.

---

## Concurrency and Consistency
### Consistency Problems & Solutions
**Definition**: Ensuring that all users see the same correct version of data.

1. **Strong Consistency**: Every read retrieves the latest write immediately.
2. **Eventual Consistency**: Data may initially be stale but will become consistent over time.

**Techniques to Ensure Consistency**:
- **Database Transactions (ACID)**: Ensuring strict data integrity.
- **Distributed Consensus Protocols**: Utilizing protocols like Paxos or Raft to manage distributed systems.

### Concurrency Problems & Solutions
**Definition**: Issues arising when multiple processes access shared resources simultaneously.

1. **Race Condition**: Multiple processes modify shared data concurrently.
   - **Solution**: Use appropriate locking mechanisms.

2. **Lost Updates**: One update can overwrite another due to concurrent access.
   - **Solution**: Implement versioning or timestamps.

### Handling Concurrency
- **Pessimistic Locking**: Locks a resource exclusively.
- **Optimistic Locking**: Users access resources with checks for conflicts before committing changes.

---

## Locking Mechanisms
### Pessimistic Locking
**Exclusive Locking** allows only one user access at a time, protecting data integrity.
```sql
BEGIN;
SELECT * FROM bids WHERE auction_id = ? FOR UPDATE;
COMMIT;
```

### Optimistic Locking
This mechanism permits reading without locking but requires checks for changes prior to updates.

---

## Utilizing Redis for Caching
### Implementation
Use Redis to store the maximum bid of each auction for rapid access and enhanced performance.

**Example of Redis Integration**:
```python
import redis

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, db=0)

# Store maximum bid
r.set('auction:1:max_bid', 100)

# Retrieve the maximum bid
max_bid = r.get('auction:1:max_bid')
```

### Advantages
- Reduces load on the database.
- Quickens the retrieval of maximum bid data.

### Disadvantages
- Potential inconsistencies can arise if Redis and the database are out of sync.

---

## Trade-offs of System Design
- **Performance vs. Consistency**: A balance must be maintained; faster responses may introduce inconsistencies.
- **Locking Trade-offs**: Heavy locking can reduce performance due to limited concurrency.
- **Caching Benefits**: Although Redis caching speeds up access times, it poses risks regarding data consistency.

---

## Schema Design
### Auction Table Schema
| Column Name       | Data Type  | Description                      |
|-------------------|------------|----------------------------------|
| id                | INT        | Primary key                      |
| item_id           | INT        | Foreign key referencing Item      |
| start_date        | DATETIME   | Start date of auction            |
| end_date          | DATETIME   | End date of auction              |
| starting_price    | DECIMAL    | Starting price of auction items   |
| user_id           | INT        | ID of seller user                |

### Bid Table Schema
| Column Name       | Data Type  | Description                      |
|-------------------|------------|----------------------------------|
| id                | INT        | Primary key                      |
| auction_id        | INT        | Foreign key referencing Auction    |
| user_id           | INT        | ID of bidder user                |
| bid_price         | DECIMAL    | Price of bid                     |
| created_at        | DATETIME   | Timestamp of bid                 |
| status            | VARCHAR    | Status: accepted or rejected      |

---

## Example SQL Code
### Insert New Bid
```sql
INSERT INTO bids (auction_id, user_id, bid_price, created_at, status)
VALUES (1, 102, 110, NOW(), 'accepted');
```

### Update Auction with Maximum Bid
```sql
UPDATE auctions
SET max_bid = (SELECT MAX(bid_price) FROM bids WHERE auction_id = 1)
WHERE auction_id = 1;
```

---

## Flowchart Illustration
```mermaid
graph TD;
    A[User Places Bid] --> B{Check Redis Cache}
    B -- Yes --> C[Compare Bid Amount]
    C -->|Accepted| D[Update Redis and DB]
    C -->|Rejected| E[Reject Bid]
    B -- No --> F[Directly Access DB]
```

---
Welcome to the Markdown Notes Generator!

Here are your notes:

Certainly! Below is a comprehensive flowchart that consolidates all elements discussed in the notes for the eBay auction system case study, including the interactions and processes for posting items, placing bids, maintaining consistency, and utilizing caching mechanisms.

## Full Flowchart of eBay Auction System

```mermaid
graph TD;

    A[User Interface] --> B[Post Auction Item]
    B --> C{Validate Input}
    C -->|Valid| D[Store Auction Data]
    D --> E[Notify User: Auction Posted]

    A --> F[Post Bid]
    F --> G{Check Auction Status}
    G -->|Auction Active| H[Check Redis Cache]
    H --> I{Is Bid Higher?}
    I -->|Yes| J[Update Bid in Redis]
    J --> K[Insert Bid into Database]
    K --> L[Notify User: Bid Accepted]
    I -->|No| M[Reject Bid: Notify User]

    G -->|Auction Inactive| N[Notify User: Auction Closed]

    A --> O[View Current Highest Bid]
    O --> P{Access Redis Cache?}
    P -->|Yes| Q[Get Highest Bid from Redis]
    P -->|No| R[Query Database for Highest Bid]
    
    R --> S[Return Highest Bid Details]
    Q --> S
    
    I --> T{Concurrency Conflict?}
    T -->|Yes| U[Check for Latest Bid in DB]
    U --> V[Allow Update Based on Latest Bid]

    subgraph Additional Processes
        W[Fault Tolerance Measures]
        X[Durability Ensured]
        Y[Scalability Considerations]
    end

    E --> W
    L --> X
    S --> Y
```

### Explanation of Flowchart Components:
1. **User Interaction**:
   - Users can perform actions such as posting auctions and bids or viewing the current highest bid.

2. **Posting an Auction**:
   - The system validates input, stores auction data, and notifies the user.

3. **Placing a Bid**:
   - On placing a bid, the system checks the auction status and then the current highest bid in the Redis cache.
   - If the new bid amount is higher than the current highest bid, it updates the Redis cache and the database.

4. **Concurrency Handling**:
   - Checks for concurrency conflicts to ensure no conflicting updates occur.
   - Uses mechanisms to resolve conflicts based on the latest data.

5. **Viewing Current Highest Bid**:
   - The system allows users to view the highest bid either from the Redis cache or directly from the database.

6. **Fault Tolerance and Durability**:
   - The system incorporates measures to ensure fault tolerance and data durability, maintaining responsiveness and reliability.

7. **Scalability**:
   - Ensures the system can handle significant user loads effectively.

---
## Full Flowchart of eBay Auction System Without Redis

```mermaid
graph TD;

    A[User Interface] --> B[Post Auction Item]
    B --> C{Validate Input}
    C -->|Valid| D[Store Auction Data in Database]
    D --> E[Notify User: Auction Posted]

    A --> F[Post Bid]
    F --> G{Check Auction Status}
    G -->|Auction Active| H[Lock Auction for Updates]
    H --> I[Get Current Highest Bid from DB]
    I --> J{Is Bid Higher?}
    J -->|Yes| K[Insert Bid into Database]
    K --> L[Notify User: Bid Accepted]
    J -->|No| M[Reject Bid: Notify User]

    G -->|Auction Inactive| N[Notify User: Auction Closed]

    A --> O[View Current Highest Bid]
    O --> P{Lock Auction for Viewing}
    P --> Q[Retrieve Highest Bid from DB]
    
    Q --> R{Filter Results}
    R --> S[Return Highest Bid Details]

    T{Concurrency Conflict?}
    T -->|Yes| U[Check for Latest Bid in DB]
    U --> V[Allow Update Based on Latest Bid]

    subgraph Additional Processes
        W[Fault Tolerance Measures]
        X[Durability Ensured]
        Y[Scalability Considerations]
    end

    E --> W
    L --> X
    S --> Y
```

### Explanation of Flowchart Components:
1. **User Interaction**: Users interact through the interface to post auctions and bids or view the highest bid.
2. **Posting an Auction**: Validates input, stores auction details in the database, and notifies the user.
3. **Placing a Bid**: Upon placing a bid:
   - The auction is locked to prevent concurrent modifications.
   - Current highest bid is fetched from the database.
   - The system checks if the new bid is higher.
   - If accepted, the bid is stored, and the user is notified. Otherwise, rejection occurs.
4. **Viewing Current Highest Bid**: 
   - Requires locking the auction to retrieve the highest bid directly from the database.
5. **Concurrency Handling**: The system checks for conflicts during updates and ensures accuracy of the bids being placed.
6. **Fault Tolerance and Durability**: Overall design incorporates resilience and reliability under varying loads.

---

## Summary
These notes encapsulate the design of an auction system similar to eBay, highlighting functionalities, required technologies, and strategies for maintaining consistency and managing concurrency. The focus on fault tolerance and caching mechanisms illustrates how such systems can efficiently handle high volumes of transactions and requests, maintaining responsiveness and reliability.
